In [1]:
import numpy as np 
import pandas as pd
from openbb_terminal.sdk import openbb
import plotly.graph_objects as go
from lib.strategies.channel import ChannelBreakoutStrategy
from scipy import stats

Forecasting Toolkit is disabled. To use the Forecasting features please install the toolkit following the instructions here: https://my.openbb.co/app/sdk/installation

Portfolio Optimization Toolkit is disabled. To use the Optimization features please install the toolkit following the instructions here: https://my.openbb.co/app/sdk/installation

### Load data 

In [2]:
start_date = "2018-01-05"
end_date = "2018-12-05"

df = openbb.stocks.load(symbol="AMD", start_date=start_date, end_date=end_date)
df.reset_index(inplace=True)

Loading Daily data for AMD with starting period 2018-01-05.

In [3]:
df.head()

,date,Open,High,Low,Close,Adj Close,Volume,Dividends,Stock Splits
0,2018-01-05,12.19,12.22,11.66,11.88,11.88,63808900,0.0,0.0
1,2018-01-08,12.01,12.30,11.85,12.28,12.28,63346000,0.0,0.0
2,2018-01-09,12.00,12.02,11.71,11.82,11.82,62560900,0.0,0.0
3,2018-01-10,11.63,12.15,11.56,11.96,11.96,52561200,0.0,0.0
4,2018-01-11,12.07,12.19,11.90,12.14,12.14,38354900,0.0,0.0


In [4]:
fig = go.Figure(data=[go.Candlestick(x=df.date,
                open=df['Open'],
                high=df['High'],
                low=df['Low'],
                close=df['Close'])])

fig.show()


In [5]:
window_size = 1
backcandles = 15 
channel_window_size = 3

cb_strategy = ChannelBreakoutStrategy(window_size, 0.2)

In [6]:
low_idx = [1, 2, 3] 
low_idx[-2:]

[2, 3]

In [7]:
df = cb_strategy.process_dataframe(df, channel_window_size, backcandles)

Breakout counts : 0.0    178
2.0     27
1.0     26
Name: breakout_type, dtype: int64


In [9]:
cb_strategy.visualize_candle(df, 160, channel_window_size=channel_window_size, num_backcandles=backcandles,optimize=False)

highs Int64Index([148, 152], dtype='int64')
low Int64Index([142, 144, 150, 153, 155], dtype='int64')
0.06900629109764701 0.12750005722045898
parallel ratio :  0.14883429116529562
low r2 :  0.665129377276237
high r2 :  1.0


In [19]:
df.iloc[20]

date                 2018-02-05 00:00:00
Open                               12.05
High                               12.18
Low                                10.93
Close                              11.57
Adj Close                          11.57
Volume                         116568900
Dividends                            0.0
Stock Splits                         0.0
pivot_type                             1
plot_pivot_val                     10.83
breakout_type                        1.0
num_backcandles                     15.0
plot_breakout_val                10.9299
Name: 20, dtype: object

In [20]:
from backtesting import Strategy, Backtest
 
def SIGNAL(): 
    return df.breakout_type

class MyStrat(Strategy):
    mysize = 10000
    def init(self):
        super().init()
        self.signal = self.I(SIGNAL)

    def next(self):
        super().next()
        TPSLRatio = 1
        perc = 0.03
    
        if self.signal[-1] == 2 and len(self.trades)==0:
            sl = self.data.Close[-1]-self.data.Close[-1]*perc
            sldiff = abs(sl-self.data.Close[-1])
            tp = self.data.Close[-1]+sldiff*TPSLRatio
            # print("buy")
            # print("prev signal : ", self.data.breakout_type[-1])
            # print("current candle :", self.data)
            # print("tp :", tp)
            # print("sl : ", sl)
            self.buy(sl=sl, tp=tp)
            # print(self.trades)
            # print()

        elif self.signal[-1] == 1 and len(self.trades)==0:         
            sl = self.data.Close[-1]+self.data.Close[-1]*perc
            sldiff = abs(sl-self.data.Close[-1])
            tp = self.data.Close[-1]-sldiff*TPSLRatio
            # print("Sell")
            # print("prev signal : ", self.data.breakout_type[-1])
            # print("current candle :", self.data)
            # print("tp :", tp)
            # print("sl : ", sl)
            self.sell(sl=sl, tp=tp)
            # print(self.trades)
            # print()

bt = Backtest(df, MyStrat, cash=10000, margin=1/5)
stat, closed_trades = bt.run()
stat

Start                                     0.0
End                                     230.0
Duration                                230.0
Exposure Time [%]                   28.571429
Equity Final [$]                  5467.479295
Equity Peak [$]                  23202.160603
Return [%]                         -45.325207
Buy & Hold Return [%]               77.777783
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                  -76.475669
Avg. Drawdown [%]                  -21.140984
Max. Drawdown Duration                   92.0
Avg. Drawdown Duration              19.555556
# Trades                                 37.0
Win Rate [%]                        43.243243
Best Trade [%]                       4.154001
Worst Trade [%]                     -5.276078
Avg. Trade [%]                    

In [21]:
bt.plot()

GridPlot(id='p1326', ...)

### Gathering data for machine learning model

In [ ]:
ema_window = 10
df['EMA_Close'] = df['Close'].ewm(span=ema_window, adjust=False).mean()
df['EMA_Volume'] = df['Volume'].ewm(span=ema_window, adjust=False).mean()

X = []
y = []
future_data = []
# 1 : long , 0 : short
position_type = []
sls = [] 
tps = []

TPSLRatio = 1.05
perc = 0.03
future_param = 4
lag_window = 21 

df_ml = df.copy()
for i in range(1, lag_window):
    df[f'lag_Close_{i}'] = df['Close'].shift(i)


for trade in closed_trades:

    # skip candles with no ancestors
    if trade.entry_bar - backcandles - channel_window_size < 0: 
        continue
    
    breakout_close_price = df_ml.iloc[trade.entry_bar - 1].Close
    # print("close price : ", breakout_close_price)

    # x = df_ml.iloc[trade.entry_bar - backcandles - channel_window_size:trade.entry_bar + 1]
    x = df_ml.iloc[trade.entry_bar]
    future = df_ml.iloc[trade.entry_bar + 1:trade.entry_bar + 1 + future_param]

    if trade.is_long:
        sl = breakout_close_price - breakout_close_price * perc
        sldiff = abs(sl - breakout_close_price)
        tp = breakout_close_price + sldiff * TPSLRatio
        if (future.High >= tp).sum() > 0:
            target = 1 
        else: 
            target = 0 

    elif trade.is_short: 
        sl = breakout_close_price + breakout_close_price * perc
        sldiff = abs(sl - breakout_close_price)
        tp = breakout_close_price - sldiff*TPSLRatio
        if (future.Low <= tp).sum() > 0:
            target = 1 
        else: 
            target = 0 

    future_data.append(future)
    X.append(x) 
    y.append(target)
    position_type.append(trade.is_long)
    tps.append(tp) 
    sls.append(sl)

    # print("Position : ", "long" if trade.is_long else "short")
    

#### Visualize dataset

In [ ]:

# for i in range(len(X)):
        
#         layout = go.Layout(
#                 autosize=True,
#                 title_text=f"{'Long' if position_type[i] else 'Short'} -  Label = {y[i]}", 
#                 font=dict(color="green" if y[i] == 1 else "red")
#                 )

#         fig = go.Figure(data=[
#                 go.Scatter(x=X[i].reset_index().index, y=[X[i]['Close']], name="breakout"), 


#                 go.Candlestick(x=future_data[i].reset_index().index,
#                 open=future_data[i]['Open'],
#                 high=future_data[i]['High'],
#                 low=future_data[i]['Low'],
#                 close=future_data[i]['Close'],
#                 name="future data"
#                 ), 

#                 go.Scatter(x=list(range(4)), y=[tps[i]] * 4, mode="lines", name="tp", line=dict(color="green"))
#                 ], 
#         layout=layout)

#         fig.show()

In [78]:
len(X)

71